# Visualize memory

In [27]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [28]:
%%capture

import gensim.downloader as api
from IPython.display import Markdown as md

from lib.memory import DSDM
from lib.utils import cleanup, configs, inference, learning, preprocess, utils

import numpy as np
import pandas as pd
import pickle
import plotly.express as px
from sklearn.manifold import TSNE
import subprocess

In [29]:
display(md('#### Available memories:'))
print(f"{subprocess.getoutput('ls ./memories/ | nl')}")

#### Available memories:

     1	memory_2023-08-13 12:16:55.806391.pkl
     2	memory_2023-08-13 12:34:54.694413.pkl
     3	memory_2023-08-13 15:39:34.086703.pkl
     4	memory_2023-08-15 01:02:10.640405.pkl
     5	memory_2023-08-15 01:02:45.620262.pkl
     6	memory_2023-08-15 17:19:11.145332.pkl
     7	memory_2023-08-15 17:29:22.393839.pkl
     8	memory_2023-08-19 20:03:41.994486.pkl
     9	memory_2023-08-19 20:48:41.219584.pkl
    10	memory_2023-08-20 05:28:38.340643.pkl
    11	memory_2023-08-20 05:48:55.214127.pkl
    12	memory_2023-08-20 05:49:14.943474.pkl
    13	method1
    14	method2


In [30]:
#number = int(input())
command = "ls ./{0}/ | sed '{1}!d'"

# Load cleanup and memory
# cleanup = pickle.load(open(f"cleanups/{subprocess.getoutput(command.format('cleanups', number))}",'rb'))
# memory = pickle.load(open(f"memories/{subprocess.getoutput(command.format('memories', number))}",'rb'))

cleanup = pickle.load(open(f"cleanups/method2/cleanup_2023-09-09 23-06-41-117607.pkl",'rb'))
memory = pickle.load(open(f"memories/method2/memory_2023-09-09 23-06-41-117607.pkl",'rb'))

### Memory stats

In [31]:
len(memory.addresses)

38322

In [32]:
memory.ema_time_period

100000

In [33]:
memory.chunk_size

In [34]:
memory.n_updates

35095

In [35]:
memory.n_expansions

38322

In [36]:
memory.removed_duplicates

0

In [37]:
memory.n_deletions

0

In [38]:
print(f"Memory size: {len(memory.addresses)}")

Memory size: 38322


In [39]:
%%capture

# Load pre-trained word embeddings (Word2Vec in this example)
word_vectors = api.load("word2vec-google-news-300")

In [40]:
%%capture
#concepts = ["university", "universities"]#, 'days', 'day', 'record', 'stage']

address_embeddings = []
address_concepts = []
addresses = []
bins = []
chunk_scores = []

for idx, address in enumerate(memory.addresses[0:1000]):
    tokens = inference.get_most_similar_HVs(inference.get_similarities_to_atomic_set(address, cleanup))
    embeddings = [word_vectors[word] for word in tokens if word in word_vectors]
    #if len(set(concepts).intersection(set(tokens))) > 0:
    if embeddings:
        addresses.append(idx)
        bins.append(memory.scores[idx, 1].item())
        chunk_scores.append(memory.scores[idx, 0].item())
        address_concepts.append(" ".join(tokens))
        address_embeddings.append(sum(embeddings) / len(embeddings))

In [41]:
reduced_embeddings = TSNE(n_components=2, random_state=42, perplexity=2).fit_transform(np.array(address_embeddings))

df = pd.DataFrame(reduced_embeddings, columns=["Dimension 1", "Dimension 2"])
df["Address"] = addresses
df["Chunk"] = address_concepts
df['Bin'] = bins
df['Chunk-score'] = chunk_scores

In [42]:
fig = px.scatter(
    df, x="Dimension 1", y="Dimension 2",
    text="Chunk", hover_data=["Address", "Bin", "Chunk-score"],
    title="Memory concepts"
)
fig.show()

In [53]:
memory.scores[294]

tensor([0.6500, 0.0401], device='cuda:0', dtype=torch.float64)

In [49]:
addresses = np.random.randint(0, len(memory.addresses), size=70)
# addresses = np.argwhere((memory.chunk_scores > 0.97).cpu().detach().numpy().flatten()).flatten()
#addresses = np.argwhere((memory.scores[:, 1] > 4).cpu().detach().numpy().flatten()).flatten()
# addresses = [3, 4]

for address in addresses:
    display(md(f"### Address {address}"))
    address_sims_df = inference.get_similarities_to_atomic_set(
            memory.addresses[address],
            cleanup,
    )
    display(address_sims_df)

### Address 7345

,token,similarity
0,instituted,0.721803
1,have,0.721803
2,rather,0.120531
3,curtain,0.120531
4,jian,0.112219
5,sharp,0.103906
6,ranked,0.103906
7,bars,0.101136
8,orbit,0.098365
9,empress,0.098365


### Address 27462

,token,similarity
0,at,0.589551
1,as,0.587274
2,doses,0.580446
3,harpsichord,0.119504
4,leaf,0.113813
5,possession,0.108122
6,showed,0.104708
7,hearing,0.102432
8,1865,0.102432
9,depicted,0.102432


### Address 36047

...

,token,similarity
0,months,0.721803
1,five,0.721803
2,festival,0.112219
3,smile,0.112219
4,invasion,0.109448
5,suggestion,0.109448
6,visited,0.103906
7,dieter,0.101136
8,attended,0.101136
9,prc,0.101136


### Address 36362

,token,similarity
0,contains,0.593153
1,pictures,0.575809
2,rude,0.560778
3,expansion,0.129499
4,bullying,0.122562
5,report,0.114468
6,playboy,0.105218
7,computers,0.101749
8,arranger,0.098281
9,ana,0.097124


### Address 10876

,token,similarity
0,contains,0.721803
1,which,0.721803
2,relegated,0.117761
3,whatever,0.114990
4,1816,0.114990
5,expansion,0.112219
6,promised,0.106677
7,1971,0.106677
8,streams,0.103906
9,reconciliation,0.103906
